# Interactive Molecular Dynamics with OpenMM, ASE and NanoVer

In this notebook, we run a pre-prepared OpenMM simulation of methane molecule and a carbon nanotube using ASE, serving it for interactive molecular dynamics using NanoVer.

We'll then connect NanoVer's NGLView client to it and apply some interactive forces directly from the notebook

In [1]:
from nanover.omni import OmniRunner
from nanover.omni.ase_omm import ASEOpenMMSimulation

## Run the NanoVer server

First, we need to create an `ASEOpenMMSimulation` for our methane and nanotube system. 

Let's do so by loading in a pre-prepared NanoVer OpenMM XML file, which contains:
- the PDB file
- the OpenMM XML file, containing:
  - the System
  - the Integrator

This format ensures that the whole simulation can be specified in a single file. 

The `ASEOpenMMSimulation` class takes the pre-prepared input file and creates a simulation object that can then be served for interactive molecular dynamics using OpenMM via ASE. For an example of a more advanced setup, see [our neuraminidase notebook](./ase_openmm_neuraminidase.ipynb).

In [2]:
from nanover.openmm.serializer import deserialize_simulation

input_xml = "openmm_files/nanotube.xml"
with open(input_xml, "r") as infile:
    omm_simulation = deserialize_simulation(infile)

nanotube_simulation = ASEOpenMMSimulation.from_simulation(omm_simulation)

Now we have our simulation ready, we can set up our NanoVer server to run the simulation. We run interactive ASE-OpenMM simulations in NanoVer using the `OmniRunner` class, which is in charge of serving and running the simulation.

In [3]:
# This will just automatically close previous instances of the runner if you re-run this cell multiple times
try:
    imd_runner.close()
except NameError:
    pass
imd_runner = OmniRunner.with_basic_server(nanotube_simulation, name="nanotube-ase-omm-server")
imd_runner.next()
imd_runner.pause()

In [4]:
print(f'{imd_runner.app_server.name}: serving at {imd_runner.app_server.address}:{imd_runner.app_server.port}')

nanotube-ase-omm-server: serving at [::]:38801


As we are running the OpenMM simulation via ASE, we can use the ASE functionality to run a few simulation steps to check that everything is working:

In [5]:
nanotube_simulation.dynamics.run(10)

True

In [6]:
nanotube_simulation.dynamics.get_number_of_steps()

10

In [7]:
# Note that this is the energy in eV, not kJ mol-1 (the standard units of ASE are different to those of OpenMM and NanoVer)
nanotube_simulation.dynamics.atoms.get_potential_energy()

28.195747377190987

Now, let's leave the dynamics running dynamics in a background thread

In [8]:
imd_runner.play()

In [9]:
nanotube_simulation.dynamics.get_time()

0.9822694788464064

The server is running, so you can connect to it in VR and you'll see something like this:

![nanotube](./images/nanover_nanotube.png)



# Visualizing the simulation with NGLView 

One type of python client provided by NanoVer is the `NGLClient`, which enables visualisation of the trajectory of the molecular system with [NGLView](https://github.com/arose/nglview).

Below, we use this client to connect to the server to visualise the methane & nanotube system.

**Note**: Your browser needs to support WebGL (try Firefox or Edge)

In [10]:
from nanover.nglview import NGLClient

In [11]:
client = NGLClient.connect_to_single_server(port=imd_runner.app_server.port)

In [ ]:
client.view

## Apply an Interactive Force 

Generally, you probably want to apply forces from the VR app, NanoVer iMD - but you can do it from python! 

First, let's apply a couple of forces to hold to nanotube in place (there are better ways to do this) 

In [13]:
from nanover.imd.particle_interaction import ParticleInteraction
import numpy as np

First, we ask the server for an ID for our interactions

In [14]:
anchor1_id = client.start_interaction()

In [15]:
anchor2_id = client.start_interaction()

Now, we can create two interactions, one at each end of the nanotube, and assign a position. 
You can repeatedly run the cell below, changing the positions or strength of the interactions. See if you can catch the methane! 

**Disclaimer**: This is *much* easier in VR.

In [16]:
first_position = np.array((10,1,0)) # nanometers!
second_position = np.array((10,1,0))
anchor1 = ParticleInteraction(position=first_position, scale=10, particles=(0,), interaction_type='spring')
anchor2 = ParticleInteraction(position=second_position, scale=10, particles=(59,), interaction_type='spring')
client.update_interaction(anchor1_id, anchor1)
client.update_interaction(anchor2_id, anchor2)

In [17]:
client.stop_interaction(anchor1_id)

True

In [18]:
client.stop_interaction(anchor2_id)

True

As always, we should clean up after ourselves:

# Close the Server

In [19]:
imd_runner.close()

# Next Steps

* The [NGLViewer notebook](nanover_nglview.ipynb) is a smaller notebook designed for visualizing your own simulations.
* Set up an OpenMM simulation with an AMBER file for a [protein-ligand system](ase_openmm_neuraminidase.ipynb) and simulate it in NanoVer
* Set up a simulation of a [graphene sheet](ase_openmm_graphene.ipynb) with parameters than can be controlled from the jupyter notebook.
* Visualize an [LSD receptor in a membrane](../mdanalysis/mdanalysis_lsd.ipynb) structure with MDAnalysis and NanoVer 